Read in packages

In [2]:
import pandas as pd
import os
import numpy as np
import xarray as xr
import rioxarray
import glob
import random
import geopandas as gpd
from sklearn.utils import shuffle
from MightyMosaic import MightyMosaic
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold


In [15]:
# Load your data
df = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_poly_check_ee.csv')
df = df.rename(columns={'Image': 'ID'})
df = df[df['ID'].isin(['29690000000000-0000000000', '29690000000000-0000023296']) == False]
df['ID'] = df['ID'].astype(int)

# Load shapefile data and merge with main dataframe
in_shape = gpd.read_file('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_polygons.shp')
in_shape['geometry'] = in_shape['geometry'].apply(lambda geom: geom.wkt)
in_shape = pd.DataFrame(in_shape[['Year', 'ID', 'area']])
df = pd.merge(in_shape, df, on='ID', how='inner')
df['Year'] = df['Year'].astype(int)
df = df[df['Keep'] == 'Yes']

df = df.reset_index(drop = True)

# Initialize KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Preparing indices for entire data shuffling
df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Split data into 5 folds
fold = 0
results = []
for train_val_index, test_index in kf.split(df_shuffled):
    # Extract test set (10% of total)
    test_df = df_shuffled.iloc[test_index]

    # Extract the remaining data for this fold
    train_val_df = df_shuffled.iloc[train_val_index]

    # Further split remaining data into training (80% of total, 88.89% of remaining) and validation (10% of total, 11.11% of remaining)
    train_df, val_df = train_test_split(train_val_df, test_size=0.125 * 2, random_state=42)  # 0.125 * 0.9 = 0.1125 ~ 10% of total

    # Save each fold's datasets to CSV files
    train_df.to_csv(f'/explore/nobackup/people/spotter5/cnn_mapping/Russia/train_fold_{fold}.csv', index=False)
    val_df.to_csv(f'/explore/nobackup/people/spotter5/cnn_mapping/Russia/val_fold_{fold}.csv', index=False)
    test_df.to_csv(f'/explore/nobackup/people/spotter5/cnn_mapping/Russia/test_fold_{fold}.csv', index=False)


    fold += 1
    print(f"Fold {fold}: Train {train_df.shape}, Validation {val_df.shape}, Test {test_df.shape}")
    results.append((train_df.shape, val_df.shape, test_df.shape))

# Output the shapes to verify the splits are as expected
# for i, (train, val, test) in enumerate(results, 1):
#     print(f"Fold {i}: Train {train}, Validation {val}, Test {test}")

Fold 1: Train (4644, 4), Validation (1548, 4), Test (1548, 4)
Fold 2: Train (4644, 4), Validation (1548, 4), Test (1548, 4)
Fold 3: Train (4644, 4), Validation (1548, 4), Test (1548, 4)
Fold 4: Train (4644, 4), Validation (1548, 4), Test (1548, 4)
Fold 5: Train (4644, 4), Validation (1548, 4), Test (1548, 4)
Fold 1: Train (4644, 4), Validation (1548, 4), Test (1548, 4)
Fold 2: Train (4644, 4), Validation (1548, 4), Test (1548, 4)
Fold 3: Train (4644, 4), Validation (1548, 4), Test (1548, 4)
Fold 4: Train (4644, 4), Validation (1548, 4), Test (1548, 4)
Fold 5: Train (4644, 4), Validation (1548, 4), Test (1548, 4)


/tmp/ipykernel_2308571/2431109095.py:9: UserWarning: Geometry column does not contain geometry.
  in_shape['geometry'] = in_shape['geometry'].apply(lambda geom: geom.wkt)


In [16]:
train_df

,Year,ID,area,Keep
4506,2008,7174,311.59000,Yes
3997,2001,709,28.64330,Yes
4055,2014,14052,49.40180,Yes
6264,2015,14960,67.23170,Yes
7605,2018,17914,10.11260,Yes
...,...,...,...,...
4695,2002,1420,748.96800,Yes
6497,2018,18663,15.57390,Yes
6536,2006,5478,3.89719,Yes
6730,2012,12118,19.50430,Yes


In [20]:
test_id = test_df['ID']
train_id = train_df['ID']
val_id = val_df['ID']

#
common_elements = set(train_id).intersection(val_id)

# Convert the set back to a list, if needed
common_elements_list = list(common_elements)

print(common_elements_list)

[]
